In [ ]:
!pip install xgboost
!pip install catboost
!pip install scikit-learn
!pip install tqdm
!pip install numpy
!pip install pickle

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from tqdm import tqdm
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
import pickle
import os


In [ ]:
# Рабочая директория на Kaggle
base_directory = '/kaggle/working'

# Перебираем каждый магазин и создаем папку с его названием
for shop_name in unique_shops:
    shop_directory = os.path.join(base_directory, shop_name)

    # Проверяем, существует ли папка, и если нет, то создаем её
    if not os.path.exists(shop_directory):
        os.makedirs(shop_directory)

    print(f'Создана папка: {shop_directory}')

In [ ]:
nn_data = final_merged_filter.copy()

In [ ]:
nn_columns = ['st_id',
              'pr_sku_id',
              'pr_sales_in_units',
              'holiday',
              'year','cos_day',
              'cos_month',
              'cos_day_of_week',
              'cos_season',
              'before_holidays_n_days',
              'after_holidays_n_days']


In [ ]:
nn_data = nn_data[nn_columns]

In [ ]:

base_directory = '/kaggle/working'

contents = os.listdir(base_directory)

# Отфильтруем только папки
folders = [item for item in contents if os.path.isdir(os.path.join(base_directory, item))]

# Выводим список папок
print("Список папок в рабочей директории:")
for folder in folders:
    print(folder)


In [ ]:
def wape(y_true: np.array, y_pred: np.array):
    return np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true))

# Словарь для хранения результатов для каждой модели
results = {}
mae_lst=[]

In [ ]:
unique_shops = nn_data['st_id'].unique()
unique_skus = nn_data['pr_sku_id'].unique()

In [ ]:
%%time



# Цикл по каждому магазину
for shop_id in tqdm(unique_shops, desc="Магазины"):
    # Фильтруем данные для текущего магазина
    shop_data = nn_data[nn_data['st_id'] == shop_id]
    # Создаем путь к директории для сохранения моделей, где shop_id - идентификатор магазина
    model_dir = os.path.join(os.getcwd(), shop_id)
    # Цикл по каждому товару в магазине
    for sku_id in tqdm(unique_skus, desc="Товары"):
        # Фильтруем данные для текущего товара
        subset = shop_data[shop_data['pr_sku_id'] == sku_id]

        # Извлекаем признаки и целевую переменную для данной комбинации
        features = subset.drop(['st_id', 'pr_sku_id', 'pr_sales_in_units'], axis=1)
        target = subset['pr_sales_in_units']

        # Пропускаем обработку, если количество данных меньше 7
        if len(features) <= 6:
            continue

        # Логарифмическое преобразование целевой переменной
        target_log = np.log1p(target)

        # Создаем список моделей для ансамбля
        models = [LinearRegression(), RandomForestRegressor(), GradientBoostingRegressor()]
        # Создаем список моделей для ансамбля
        models = [
            ('linear', LinearRegression()),
            ('random_forest', RandomForestRegressor()),
            ('gradient_boosting', GradientBoostingRegressor())
        ]

        # Создаем ансамбль из моделей с помощью VotingRegressor
        ensemble_model_1 = VotingRegressor(estimators=models)

        # Массив для хранения предсказаний ансамбля
        ensemble_predictions = np.zeros(len(target_log))

        # Количество разбиений и TimeSeriesSplit
        n_splits = 5
        tscv = TimeSeriesSplit(n_splits=n_splits)

        # Цикл по каждому разбиению в кросс-валидации
        for train_index, val_index in tscv.split(features):
            X_train_fold, X_val_fold = features.iloc[train_index], features.iloc[val_index]
            y_train_fold, y_val_fold = target_log.iloc[train_index], target_log.iloc[val_index]

            # Обучаем ансамбль на тренировочных данных
            ensemble_model_1.fit(X_train_fold, y_train_fold)

            # Предсказания на валидационных данных
            val_predictions = ensemble_model_1.predict(X_val_fold)

            # Сохраняем предсказания для данного разбиения
            ensemble_predictions[val_index] = val_predictions

        # Вычисляем качество ансамбля на валидационных данных
        wape_first = wape(np.log1p(target), ensemble_predictions)
        mae_first = mean_absolute_error(np.log1p(target), ensemble_predictions)
        mae_lst.append(mae_first)
        if wape_first > 1:
            # Создаем экземпляры дополнительных моделей
            xgb_model = XGBRegressor()
            svr_model = SVR()
            lr_model = LinearRegression()

            # Создаем ансамбль из дополнительных моделей
            ensemble_models = [
                ('xgboost', xgb_model),
                ('svr', svr_model),
                ('linear_regression', lr_model)
            ]

            # Цикл по каждой модели в ансамбле
            for model_name, model in ensemble_models:
                # Создаем ансамбль только с текущей моделью
                ensemble_model = VotingRegressor(estimators=[(model_name, model)])

                # Массив для хранения предсказаний для каждой модели
                predictions = np.zeros((len(target_log),))

                # Цикл по каждому разбиению в кросс-валидации
                for train_index, val_index in tscv.split(features):
                    X_train_fold, X_val_fold = features.iloc[train_index], features.iloc[val_index]
                    y_train_fold, y_val_fold = target_log.iloc[train_index], target_log.iloc[val_index]

                    # Обучаем ансамбль на тренировочных данных
                    ensemble_model.fit(X_train_fold, y_train_fold)

                    # Предсказания на валидационных данных
                    val_predictions = ensemble_model.predict(X_val_fold)

                    # Сохраняем предсказания для данной модели и данного разбиения
                    predictions[val_index] = val_predictions

                # Вычисляем качество ансамбля на валидационных данных
                wape_second = wape(np.log1p(target), predictions)

                # Если второй WAPE лучше, сохраняем результаты
                if wape_second < wape_first:
                    results[(shop_id, sku_id)] = wape_second
                    model_name = f"{sku_id}.pkl"
                    model_path = os.path.join(model_dir, model_name)
                    with open(model_path, 'wb') as model_file:
                        pickle.dump(ensemble_model, model_file)
                else:
                    results[(shop_id, sku_id)] = wape_first
                    model_name = f"{sku_id}.pkl"
                    model_path = os.path.join(model_dir, model_name)
                    with open(model_path, 'wb') as model_file:
                        pickle.dump(ensemble_model_1, model_file)
        else:
            # Сохраняем результаты в словаре
            results[(shop_id, sku_id)] = wape_first
            model_name = f"{sku_id}.pkl"
            model_path = os.path.join(model_dir, model_name)
            with open(model_path, 'wb') as model_file:
                pickle.dump(ensemble_model_1, model_file)

# Распечатайте или используйте словарь с результатами по вашему усмотрению


In [ ]:
c = 0
lst= []
for i in results.values():
    if i>1:
        c+=1
    else:
        lst.append(i)

In [ ]:
print(f'mean MAE on test: {np.mean(mae_lst)}')

In [ ]:
print(f'mean WAPE on test (Without outliers): {np.mean(list(results.values())}')